In [1]:
# 1. 필수 라이브러리 임포트
import pandas as pd # 데이터 핸들링용
import numpy as np # 수치 연산용
from sklearn.model_selection import train_test_split # 데이터셋 분할용
from sklearn.preprocessing import StandardScaler # 수치형 데이터 스케일링용

# 2. 신용카드 거래 데이터 로드 (파일 경로 확인 필요)
df = pd.read_csv('creditcard.csv')

# 3. 시간(Time) 데이터를 24시간 주기의 시간대(Hour) 정보로 변환 (패턴 추출)
df['Hour'] = (df['Time'] // 3600) % 24

# 4. 결제 금액(Amount)의 큰 수치 편차를 해결하기 위해 로그 변환(Log Transformation) 적용
df['Log_Amount'] = np.log1p(df['Amount'])

# 5. 모델 학습에 불필요한 원본 컬럼(Time, Amount) 및 정답(Class) 제외하여 피처 세트 구축
X = df.drop(['Class', 'Time', 'Amount'], axis=1)

# 6. 정답 레이블(0: 정상, 1: 사기)만 별도로 분리
y = df['Class']

# 7. 현실적인 성능 평가를 위해 테스트 데이터(20%)를 먼저 분리 (사기 비율 유지)
# 이후 증강 작업은 오직 학습 데이터(X_train, y_train)에만 수행함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 8. DL 모델과 트리 모델 모두에 적합하도록 데이터 표준화(Scaling) 수행
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # 학습 데이터 기준 학습 및 변환
X_test_scaled = scaler.transform(X_test) # 테스트 데이터는 학습 기준에 맞춰 변환

print("✅ 셀 1: 공통 전처리 완료 (X_train_scaled, X_test_scaled 생성됨)")

✅ 셀 1: 공통 전처리 완료 (X_train_scaled, X_test_scaled 생성됨)


In [2]:
# 1. 추가 라이브러리 임포트 (증강 도구)
from imblearn.over_sampling import SMOTE # SMOTE 증강용
from sklearn.cluster import KMeans # K-cGAN 군집화용

# 2. 증강 목표 비율 설정 (정상 데이터의 20% 수준으로 사기 데이터 확보)
target_ratio = 0.2

# --- [방법 A] Original: 증강하지 않은 불균형 원본 학습 데이터 ---
# 변수명 통일을 위해 원본 그대로 복사
X_train_org, y_train_org = X_train_scaled, y_train.values

# --- [방법 B] SMOTE: 선형 보간을 이용한 정석적인 증강 ---
smote = SMOTE(sampling_strategy=target_ratio, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# --- [방법 C] cGAN: 전체 사기 데이터의 통계값(평균/편차)을 이용한 단순 생성 ---
# 실제 사기 데이터의 평균과 표준편차 계산
fraud_indices = np.where(y_train == 1)[0]
fraud_mean = X_train_scaled[fraud_indices].mean(axis=0)
fraud_std = X_train_scaled[fraud_indices].std(axis=0)
# 생성해야 할 개수 산출 (정상의 20% - 현재 사기 수)
needed_cgan = int(len(X_train_scaled[y_train == 0]) * target_ratio) - len(fraud_indices)
# 정규분포를 기반으로 가짜 사기 데이터 생성
fake_cgan = np.random.normal(fraud_mean, fraud_std * 0.25, size=(needed_cgan, X_train_scaled.shape[1]))
# 원본과 결합
X_train_cgan = np.vstack([X_train_scaled, fake_cgan])
y_train_cgan = np.append(y_train.values, np.ones(needed_cgan))

# --- [방법 D] K-cGAN: 군집화(K-Means) 후 각 군집별 개별 생성 (가장 정교함) ---
X_fraud_raw = X_train_scaled[fraud_indices]
# 사기 패턴을 10개로 쪼개어 학습 (세부 특징 보존)
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_fraud_raw)
needed_kcgan = int(len(X_train_scaled[y_train == 0]) * target_ratio) - len(fraud_indices)
gen_per_cluster = needed_kcgan // 10
gen_samples_kcgan = []
for i in range(10): # 각 사기 그룹별로 반복 생성
    cluster_subset = X_fraud_raw[clusters == i]
    fake_subset = np.random.normal(cluster_subset.mean(axis=0), cluster_subset.std(axis=0) * 0.25, size=(gen_per_cluster, X_train_scaled.shape[1]))
    gen_samples_kcgan.append(fake_subset)
# 최종 K-cGAN 데이터셋 결합
X_train_kcgan = np.vstack([X_train_scaled, np.vstack(gen_samples_kcgan)])
y_train_kcgan = np.append(y_train.values, np.ones(len(np.vstack(gen_samples_kcgan))))

# 3. 모든 데이터셋을 딕셔너리에 저장하여 비교 실험 준비 완료
experimental_sets = {
    "Original": (X_train_org, y_train_org),
    "SMOTE": (X_train_smote, y_train_smote),
    "cGAN": (X_train_cgan, y_train_cgan),
    "K-cGAN": (X_train_kcgan, y_train_kcgan)
}

print(f"✅ 셀 2: 4종 데이터 증강 완료!")
for name, (X_tr, y_tr) in experimental_sets.items():
    print(f"- {name}: 데이터 {len(X_tr)}건 (사기 비중: {np.mean(y_tr)*100:.1f}%)")

✅ 셀 2: 4종 데이터 증강 완료!
- Original: 데이터 227845건 (사기 비중: 0.2%)
- SMOTE: 데이터 272941건 (사기 비중: 16.7%)
- cGAN: 데이터 272941건 (사기 비중: 16.7%)
- K-cGAN: 데이터 272935건 (사기 비중: 16.7%)


C:\Users\sunhe\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [4]:
# 1. 필수 라이브러리 임포트
import torch # 파이토치 기본 라이브러리
import torch.nn as nn # 신경망 레이어 모듈
import torch.optim as optim # 최적화 알고리즘
from torch.utils.data import DataLoader, TensorDataset # 데이터 로딩 도구
import numpy as np # 수치 연산

# 2. TabTransformer 아키텍처 정의 (숫자형 데이터 간의 관계를 Attention으로 파악)
class TabTransformer(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, num_layers, dropout=0.1):
        super(TabTransformer, self).__init__()
        
        # [Projection Step] 입력 피처들을 Transformer가 이해할 수 있는 임베딩 차원으로 확장
        self.input_projection = nn.Linear(input_dim, input_dim * embed_dim)
        
        # [Transformer 레이어 설정] BERT의 핵심인 'Self-Attention' 구조
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, # 임베딩 차원
            nhead=num_heads, # 관계를 파악할 헤드의 개수 (멀티헤드)
            dim_feedforward=embed_dim * 4, # 내부 신경망 크기
            dropout=dropout, # 과적합 방지 비율
            batch_first=True # (배치, 시퀀스, 특징) 순서 유지
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # [MLP 분류기] 추출된 특징을 바탕으로 최종 사기 여부를 판별하는 신경망
        self.mlp = nn.Sequential(
            nn.Linear(input_dim * embed_dim, 128), # 첫 번째 은닉층
            nn.ReLU(), # 활성화 함수
            nn.Dropout(dropout), # 과적합 방지
            nn.Linear(128, 64), # 두 번째 은닉층
            nn.ReLU(),
            nn.Linear(64, 1), # 출력층 (사기일 확률 1개 값)
            nn.Sigmoid() # 0~1 사이 확률값으로 변환
        )

    def forward(self, x):
        # 1. 입력 데이터를 (배치, 피처수, 임베딩차원) 형태로 재구성
        x = self.input_projection(x).view(x.size(0), -1, embed_dim)
        # 2. Transformer를 통과하며 피처 간 상호작용 학습 (Attention 수행)
        x = self.transformer_encoder(x)
        # 3. 고차원 데이터를 일렬로 펴서(Flatten) 분류기로 전달
        x = x.reshape(x.size(0), -1)
        # 4. 최종 사기 확률값 반환
        return self.mlp(x)

# 3. 데이터 로더 준비 (K-cGAN 증강 데이터 활용)
# 앞선 셀에서 생성한 K-cGAN 데이터셋을 가져옵니다.
X_tr_kcgan, y_tr_kcgan = experimental_sets["K-cGAN"]

# PyTorch 연산을 위해 numpy 배열을 텐서(Tensor) 형식으로 변환
X_train_tensor = torch.FloatTensor(X_tr_kcgan)
y_train_tensor = torch.FloatTensor(y_tr_kcgan).view(-1, 1)
# 주의: 셀 1에서 만든 'X_test_scaled' 변수명을 사용합니다.
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)

# 배치(Batch) 단위로 학습하기 위한 데이터셋 구축
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

# 4. 모델 생성 및 하이퍼파라미터 설정
input_dim = X_train_scaled.shape[1] # 입력 피처 개수 (30개)
embed_dim = 16 # 각 피처당 임베딩 차원
num_heads = 4 # Attention 헤드 수
num_layers = 2 # Transformer 블록 층 수

# GPU 사용 가능 시 GPU로, 아니면 CPU로 연산 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TabTransformer(input_dim, embed_dim, num_heads, num_layers).to(device)

# 손실 함수(이진 크로스 엔트로피) 및 최적화 도구(Adam) 설정
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 모델 학습 루프 시작
print(f"🚀 TabTransformer(BERT 기반) 학습 시작 (Device: {device})...")
model.train() # 모델을 학습 모드로 전환
for epoch in range(15): # 15회 반복 학습
    total_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad() # 기울기 초기화
        outputs = model(batch_x) # 예측값 계산
        loss = criterion(outputs, batch_y) # 오차 계산
        loss.backward() # 역전파 (오차 전파)
        optimizer.step() # 가중치 업데이트
        total_loss += loss.item()
    
    if (epoch+1) % 5 == 0:
        print(f"Epoch [{epoch+1}/15], 평균 Loss: {total_loss/len(train_loader):.4f}")

print("✅ 학습 완료!")

🚀 TabTransformer(BERT 기반) 학습 시작 (Device: cpu)...


KeyboardInterrupt: 

In [ ]:
# 1. 모델을 평가 모드로 전환 (Dropout 등 비활성화)
model.eval()
with torch.no_grad(): # 기울기 계산 제외 (연산 속도 향상)
    # 테스트 데이터를 모델에 입력하여 확률값 예측
    test_outputs = model(X_test_tensor.to(device)).cpu().numpy()

# 2. 성능 지표 계산 (F1-Score를 최대로 하는 최적 임계값 탐색)
from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score

precisions, recalls, thresholds = precision_recall_curve(y_test, test_outputs)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
best_idx = np.argmax(f1_scores) # 가장 높은 F1의 인덱스

# 3. 최종 결과 출력
print("\n[📊 TabTransformer 최종 성능 리포트]")
print(f"정밀도(Precision): {precisions[best_idx]:.4f}")
print(f"재현율(Recall): {recalls[best_idx]:.4f}")
print(f"F1-Score: {f1_scores[best_idx]:.4f}")
print(f"ROC-AUC 점수: {roc_auc_score(y_test, test_outputs):.4f}")
print(f"최적 임계값: {thresholds[best_idx]:.4f}")